In [4]:
%%writefile LayerNorm.cu
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>

__global__ void LayerNorm(const float* A, float* B, int rows, int cols) {
    // Calculate row index
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < rows) {
        // Use shared memory for row-wise computation
        extern __shared__ float shared[];
        float* row_data = shared;

        // Copy row data to shared memory
        for (int col = threadIdx.y; col < cols; col += blockDim.y) {
            row_data[col] = A[row * cols + col];
            printf("Row %d, Thread Y %d, Copying A[%d] to shared[%d]: %.2f\n", row, threadIdx.y, row * cols + col, col, row_data[col]);
        }
        __syncthreads();

        // Compute mean
        float mean = 0.0f;
        for (int col = 0; col < cols; col++) {
            mean += row_data[col];
            printf("Row %d, Thread X %d, Calculating mean: adding %.2f, current mean: %.2f\n", row, threadIdx.x, row_data[col], mean);
        }
        mean /= cols;
        printf("Row %d, Thread X %d, Final mean: %.2f\n", row, threadIdx.x, mean);

        // Compute variance
        float variance = 0.0f;
        for (int col = 0; col < cols; col++) {
            variance += (row_data[col] - mean) * (row_data[col] - mean);
            printf("Row %d, Thread X %d, Calculating variance: adding (%.2f - %.2f)^2, current variance: %.2f\n", row, threadIdx.x, row_data[col], mean, variance);
        }
        variance /= cols;
        float stddev = sqrtf(variance + 1e-7);
        printf("Row %d, Thread X %d, Final variance: %.2f, Stddev: %.2f\n", row, threadIdx.x, variance, stddev);


        // Normalize
        for (int col = threadIdx.y; col < cols; col += blockDim.y) {
            B[row * cols + col] = (row_data[col] - mean) / stddev;
            printf("Row %d, Thread Y %d, Normalizing shared[%d]: (%.2f - %.2f) / %.2f = %.2f\n", row, threadIdx.y, col, row_data[col], mean, stddev, B[row * cols + col]);
        }
    }
}

int main() {
    const int rows = 10, cols = 10;
    float *A, *B;

    // Allocate host memory
    A = (float*)malloc(rows * cols * sizeof(float));
    B = (float*)malloc(rows * cols * sizeof(float));

    // Initialize input matrix
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            A[i * cols + j] = static_cast<float>(rand()) / RAND_MAX;
        }
    }

    // Allocate device memory
    float *d_a, *d_b;
    cudaMalloc(&d_a, rows * cols * sizeof(float));
    cudaMalloc(&d_b, rows * cols * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_a, A, rows * cols * sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel
    int blocksize_x = 256;
    int blocksize_y = 1; // Adjust blocksize_y based on cols
    dim3 blockDim(blocksize_x, blocksize_y);
    dim3 gridDim((rows + blocksize_x - 1) / blocksize_x);

    size_t shared_memory_size = cols * sizeof(float);
    LayerNorm<<<gridDim, blockDim, shared_memory_size>>>(d_a, d_b, rows, cols);

    // Synchronize device
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(B, d_b, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);

    // Print results
    printf("A:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%.2f ", A[i * cols + j]);
        }
        printf("\n");
    }

    printf("\nB:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%.2f ", B[i * cols + j]);
        }
        printf("\n");
    }

    // Free memory
    cudaFree(d_a);
    cudaFree(d_b);
    free(A);
    free(B);

    return 0;
}

Writing LayerNorm.cu


In [5]:
# Compile with the specified architecture
!nvcc LayerNorm.cu -o LayerNorm -gencode arch=compute_75,code=sm_75

# Run the executable
!./LayerNorm

Row 0, Thread Y 0, Copying A[0] to shared[0]: 0.84
Row 1, Thread Y 0, Copying A[10] to shared[0]: 0.48
Row 2, Thread Y 0, Copying A[20] to shared[0]: 0.02
Row 3, Thread Y 0, Copying A[30] to shared[0]: 0.51
Row 4, Thread Y 0, Copying A[40] to shared[0]: 0.53
Row 5, Thread Y 0, Copying A[50] to shared[0]: 0.53
Row 6, Thread Y 0, Copying A[60] to shared[0]: 0.24
Row 7, Thread Y 0, Copying A[70] to shared[0]: 0.53
Row 8, Thread Y 0, Copying A[80] to shared[0]: 0.69
Row 9, Thread Y 0, Copying A[90] to shared[0]: 0.96
Row 0, Thread Y 0, Copying A[1] to shared[1]: 0.39
Row 1, Thread Y 0, Copying A[11] to shared[1]: 0.63
Row 2, Thread Y 0, Copying A[21] to shared[1]: 0.24
Row 3, Thread Y 0, Copying A[31] to shared[1]: 0.84
Row 4, Thread Y 0, Copying A[41] to shared[1]: 0.77
Row 5, Thread Y 0, Copying A[51] to shared[1]: 0.09
Row 6, Thread Y 0, Copying A[61] to shared[1]: 0.97
Row 7, Thread Y 0, Copying A[71] to shared[1]: 0.04
Row 8, Thread Y 0, Copying A[81] to shared[1]: 0.17
Row 9, Thread 